# Tokenization
Tokenization is at the heart of some of the main issues with LLMs.
1. LLMs can not spell words.
2. LLMs truggle with super simple string tasks like reversing a string.
3. Why are LLMs worse at non-English languages ?
4. 

Try out tokenization for different LLms here:

https://tiktokenizer.vercel.app/?model=codellama%2FCodeLlama-70b-hf

## Notes:
1. Space is sometimes included as part of a token
2. Digits are also tokenized.
3. Tokenization is case sensitive.
4. We have more data in english than in non-english, so english sentences have longer tokens due to this abundance. So a longer context can be fed into a model for a given number of context length.
5. GPT2 is not very good with python. One of the reasons is that the spaces are treated as different tokens. We run out of the context length. GPT4 tokenizer handles python much better, as it groups together the white spaces, which was a deliberate choice made by openAI. 
6. Increasing the vocab size is not always good, since it increases the embedding table size and at the output, it affects the performance of softmax layer.


## Unicode Encoding
- Unicode system includes almost 150, 000 characters across 161 scripts, what they look like and what integer represents them. We can access the unicode code point using the "ord" function in python.
-  Why can't we use this coding natively for tokenization?
    The vocab would be very largeThe unicode encoding keeps changing.
    The unicode encoding keeps changing.
- The unicode encoding is how the standard abstacted codes for characters are translated and stored as into sequences of bytes. UTF-8 (most common), UTF-16, UTF-32.
- UTF-8 encoding takes each of the code point and translates it into a byte stream from 1 to 4 bytes (variable length encoding). UTF-32 is fixed length.
- UTF-8 is preferred because it is backward compatible into much simpler  ASCII encoding.
- UTF-16 and UTF-32 are somewhat wasteful as they have a lot of 0's between them.
- We cannot use these encodings naively because although they would ensure a finite embedding table of size 256 and smaller output size, but we would run out of context length.
- Readings: A programmers introduction to Unicode,  UTF-8 everywhere

In [1]:
ord("f") # We can't plug in a string  here. It expects a single character. 

102

In [2]:
#list of raw bytes of utf-8 encoding 
list(("hello").encode("utf-8"))

[104, 101, 108, 108, 111]

## Byte Pair Encoding
- Iteratively find the most common "byte pair" in the given sequence.
- Replace that with a new token and add that new token to our vocabulary
- Length of sequence decreases and vocabulary size increases
- 

In [27]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
print("TEXT")
print(text)
print(f"Length of text: {len(text)}")
print("________________")


#UTF-8 Encoding 
tokens = text.encode("utf-8") #raw bytes
tokens = list(map(int, tokens))
print("TOKENS")
print(tokens)
print(f"Length of tokens: {len(tokens)}")
print("________________")




TEXT
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
Length of text: 533
________________
TOKENS
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240,

In [28]:
#Find the most frequent token
def get_stats(ids):
    token_dict = {}
    for i in range(len(ids) - 1):
        a = ids[i]
        b = ids[i + 1]
        token_dict[(a, b)] = token_dict.get((a, b), 0) + 1
    return token_dict
stats = get_stats(tokens)
most_freq_token = max(stats, key = stats.get)
print(most_freq_token)

(101, 32)


In [32]:
#Iterate over the sequence. Replace all instances of the most freq token (101, 32) by 256 (since our current vocab size is 255 
def replace_token(tokens, freq_token, name):
    seq = []
    i = 0
    while i < len(tokens) - 1:
        
        if (tokens[i], tokens[i+1]) == freq_token:
            seq.append(name)
            i += 2
        else:
            seq.append(tokens[i])
            i += 1
            
    return seq
            
        

In [33]:
my_list = replace_token(tokens, (101,32), 256)
len(my_list)

595

21